In [21]:
#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#training-on-gpu
import torch, torchvision, os, cv2
from torch.utils.data import random_split, Dataset
from torch.nn import Sigmoid, Tanh, Linear, ReLU, Sequential, Conv2d, MaxPool2d, Sigmoid, BatchNorm2d, Flatten, ConvTranspose2d
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

images = np.load('LogImagesV2.npy') #Rade the trianing data.
images = np.moveaxis(images, -1, 1) #Reshape channeL from [B, H, W, C] to [B, C, H, W]
labels = np.load('LabelsV2.npy') #Rade the trianing data. 
labels = labels.reshape(labels.shape[0],1)
# images = images.astype(np.float32)
# labels = labels.astype(np.int)

# print(labels.shape)
labels2D = np.zeros((labels.shape[0],2))
for i in range(len(labels)):
    lab = labels[i]
    if lab == 0:
        labels2D[i,0] = 1
    if lab == 1:
        labels2D[i,1] = 1
lengths = [round(len(images)*0.8), round(len(images)*0.2)]
# print(lengths)
trainImg, testImg = random_split(images, lengths ,generator=torch.random.manual_seed(42)) #Shuffle data with random seed 42 before split train and test
trainLab, testLab = random_split(labels, lengths ,generator=torch.random.manual_seed(42)) #Shuffle data with random seed 42 before split train and test
# print(images.shape)
# aa, bb = random_split(range(479), lengths, generator=torch.random.manual_seed(42))
# print(list(bb))
# print(trainImg[0].shape)
# print(trainLab[25])


trainData = [] 
for i in range(len(trainImg)):
    trainData.append([torch.tensor(trainImg[i], dtype=torch.float32), torch.tensor(trainLab[i],dtype=torch.float32)])
trainLoader = torch.utils.data.DataLoader(trainData, shuffle=True, batch_size=5)

testData = []
for i in range(len(testImg)):
    testData.append([torch.tensor(testImg[i], dtype=torch.float32), torch.tensor(testLab[i],dtype=torch.float32)])
testLoader = torch.utils.data.DataLoader(testData, shuffle=False, batch_size=5)

In [22]:
net = Sequential(
            # Defining 1st 2D convolution layer
            Conv2d(3, 16, kernel_size=3, stride=1, padding=1), #200@3 
            BatchNorm2d(16),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 2nd 2D convolution layer
            Conv2d(16, 8, kernel_size=3, stride=1, padding=1), #100@3 
            BatchNorm2d(8),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 3rd 2D convolution layer
            Conv2d(8, 4, kernel_size=3, stride=1, padding=1), #50@3 
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 4th 2D convolution layer
            Conv2d(4, 2, kernel_size=3, stride=1, padding=1), #25@3 
            BatchNorm2d(2),
            ReLU(inplace=True),
            Flatten(),
            Linear(2 * 25 * 25, 1),
            Sigmoid()
        )

net = net.cuda()
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
print(net)

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU(inplace=True)
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Conv2d(4, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU(inplace=True)
  

In [23]:
#%%time
n_epoch = 10000
for epoch in range(n_epoch):  # loop over the dataset multiple times
#epoch = 0
#while True:
    train_loss = 0.0
    for i, data in enumerate(trainLoader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].cuda(), data[1].cuda() #Reg
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        labels = torch.reshape(labels, (-1,))
        outputs = torch.reshape(outputs, (-1,))

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss += loss.item()
    logStr = 'Train epoch: %d, Loss: %.10f' % (epoch + 1, train_loss/ (i+1))
    print(logStr)
    epoch += 1

print('Finished Training')

Train epoch: 1, Loss: 0.7136603636
Train epoch: 2, Loss: 0.6686695377
Train epoch: 3, Loss: 0.6628549339
Train epoch: 4, Loss: 0.6106421479
Train epoch: 5, Loss: 0.5461075616
Train epoch: 6, Loss: 0.4993415488
Train epoch: 7, Loss: 0.4445489210
Train epoch: 8, Loss: 0.4275346731
Train epoch: 9, Loss: 0.3872258048
Train epoch: 10, Loss: 0.3089829898
Train epoch: 11, Loss: 0.3035905191
Train epoch: 12, Loss: 0.2125433378
Train epoch: 13, Loss: 0.1318485370
Train epoch: 14, Loss: 0.0951985086
Train epoch: 15, Loss: 0.0610928753
Train epoch: 16, Loss: 0.0403452826
Train epoch: 17, Loss: 0.0257721651
Train epoch: 18, Loss: 0.0191360508
Train epoch: 19, Loss: 0.0153944975
Train epoch: 20, Loss: 0.0122510612
Train epoch: 21, Loss: 0.0099636372
Train epoch: 22, Loss: 0.0088140195
Train epoch: 23, Loss: 0.0076653901
Train epoch: 24, Loss: 0.0067816340
Train epoch: 25, Loss: 0.0061806795
Train epoch: 26, Loss: 0.0056296599
Train epoch: 27, Loss: 0.0054786626
Train epoch: 28, Loss: 0.0047505464
T

Train epoch: 226, Loss: 0.0002254598
Train epoch: 227, Loss: 0.0002206329
Train epoch: 228, Loss: 0.0002134929
Train epoch: 229, Loss: 0.0002051263
Train epoch: 230, Loss: 0.0002357740
Train epoch: 231, Loss: 0.0002110806
Train epoch: 232, Loss: 0.0002215396
Train epoch: 233, Loss: 0.0002026214
Train epoch: 234, Loss: 0.0001938635
Train epoch: 235, Loss: 0.0002024199
Train epoch: 236, Loss: 0.0001967989
Train epoch: 237, Loss: 0.0001984305
Train epoch: 238, Loss: 0.0001950748
Train epoch: 239, Loss: 0.0001996272
Train epoch: 240, Loss: 0.0001971797
Train epoch: 241, Loss: 0.0001846954
Train epoch: 242, Loss: 0.0001965611
Train epoch: 243, Loss: 0.0001931730
Train epoch: 244, Loss: 0.0001888650
Train epoch: 245, Loss: 0.0001808630
Train epoch: 246, Loss: 0.0002078403
Train epoch: 247, Loss: 0.0001804770
Train epoch: 248, Loss: 0.0001848353
Train epoch: 249, Loss: 0.0001835049
Train epoch: 250, Loss: 0.0002148995
Train epoch: 251, Loss: 0.0001847854
Train epoch: 252, Loss: 0.0001800672
T

Train epoch: 448, Loss: 0.0000835788
Train epoch: 449, Loss: 0.0000911840
Train epoch: 450, Loss: 0.0000840808
Train epoch: 451, Loss: 0.0000838919
Train epoch: 452, Loss: 0.0000792895
Train epoch: 453, Loss: 0.0000856058
Train epoch: 454, Loss: 0.0000836439
Train epoch: 455, Loss: 0.0000871585
Train epoch: 456, Loss: 0.0000899556
Train epoch: 457, Loss: 0.0000963684
Train epoch: 458, Loss: 0.0000863517
Train epoch: 459, Loss: 0.0000915171
Train epoch: 460, Loss: 0.0000865246
Train epoch: 461, Loss: 0.0000855340
Train epoch: 462, Loss: 0.0000854628
Train epoch: 463, Loss: 0.0000857661
Train epoch: 464, Loss: 0.0000843434
Train epoch: 465, Loss: 0.0000911022
Train epoch: 466, Loss: 0.0000801179
Train epoch: 467, Loss: 0.0000880562
Train epoch: 468, Loss: 0.0000921756
Train epoch: 469, Loss: 0.0000845318
Train epoch: 470, Loss: 0.0000778665
Train epoch: 471, Loss: 0.0000829399
Train epoch: 472, Loss: 0.0000842619
Train epoch: 473, Loss: 0.0000915489
Train epoch: 474, Loss: 0.0000798875
T

Train epoch: 670, Loss: 0.0000518706
Train epoch: 671, Loss: 0.0000622747
Train epoch: 672, Loss: 0.0000553127
Train epoch: 673, Loss: 0.0000534460
Train epoch: 674, Loss: 0.0000595773
Train epoch: 675, Loss: 0.0000538652
Train epoch: 676, Loss: 0.0000564451
Train epoch: 677, Loss: 0.0000611517
Train epoch: 678, Loss: 0.0000529081
Train epoch: 679, Loss: 0.0000661829
Train epoch: 680, Loss: 0.0000535607
Train epoch: 681, Loss: 0.0000569008
Train epoch: 682, Loss: 0.0000535275
Train epoch: 683, Loss: 0.0000525540
Train epoch: 684, Loss: 0.0000571607
Train epoch: 685, Loss: 0.0000560221
Train epoch: 686, Loss: 0.0000513925
Train epoch: 687, Loss: 0.0000523202
Train epoch: 688, Loss: 0.0000590515
Train epoch: 689, Loss: 0.0000573946
Train epoch: 690, Loss: 0.0000534739
Train epoch: 691, Loss: 0.0000592625
Train epoch: 692, Loss: 0.0000540726
Train epoch: 693, Loss: 0.0000553624
Train epoch: 694, Loss: 0.0000598725
Train epoch: 695, Loss: 0.0000525333
Train epoch: 696, Loss: 0.0000557631
T

Train epoch: 892, Loss: 0.0000395228
Train epoch: 893, Loss: 0.0000407269
Train epoch: 894, Loss: 0.0000393417
Train epoch: 895, Loss: 0.0000373891
Train epoch: 896, Loss: 0.0000440516
Train epoch: 897, Loss: 0.0000394308
Train epoch: 898, Loss: 0.0000377525
Train epoch: 899, Loss: 0.0000383982
Train epoch: 900, Loss: 0.0000424113
Train epoch: 901, Loss: 0.0000384149
Train epoch: 902, Loss: 0.0000367734
Train epoch: 903, Loss: 0.0000398622
Train epoch: 904, Loss: 0.0000419541
Train epoch: 905, Loss: 0.0000416034
Train epoch: 906, Loss: 0.0000353586
Train epoch: 907, Loss: 0.0000442254
Train epoch: 908, Loss: 0.0000397096
Train epoch: 909, Loss: 0.0000370466
Train epoch: 910, Loss: 0.0000378174
Train epoch: 911, Loss: 0.0000398154
Train epoch: 912, Loss: 0.0000427238
Train epoch: 913, Loss: 0.0000406167
Train epoch: 914, Loss: 0.0000383567
Train epoch: 915, Loss: 0.0000372256
Train epoch: 916, Loss: 0.0000362778
Train epoch: 917, Loss: 0.0000415464
Train epoch: 918, Loss: 0.0000405058
T

...

Train epoch: 9103, Loss: 0.0000027334
Train epoch: 9104, Loss: 0.0000027481
Train epoch: 9105, Loss: 0.0000028573
Train epoch: 9106, Loss: 0.0000029246
Train epoch: 9107, Loss: 0.0000032505
Train epoch: 9108, Loss: 0.0000028353
Train epoch: 9109, Loss: 0.0000028499
Train epoch: 9110, Loss: 0.0000030617
Train epoch: 9111, Loss: 0.0000028105
Train epoch: 9112, Loss: 0.0000028823
Train epoch: 9113, Loss: 0.0000037439
Train epoch: 9114, Loss: 0.0000030367
Train epoch: 9115, Loss: 0.0000030813
Train epoch: 9116, Loss: 0.0000031137
Train epoch: 9117, Loss: 0.0000024017
Train epoch: 9118, Loss: 0.0000028273
Train epoch: 9119, Loss: 0.0000029254
Train epoch: 9120, Loss: 0.0000032076
Train epoch: 9121, Loss: 0.0000031212
Train epoch: 9122, Loss: 0.0000027881
Train epoch: 9123, Loss: 0.0000027839
Train epoch: 9124, Loss: 0.0000026704
Train epoch: 9125, Loss: 0.0000030504
Train epoch: 9126, Loss: 0.0000028214
Train epoch: 9127, Loss: 0.0000025323
Train epoch: 9128, Loss: 0.0000039981
Train epoch:

Train epoch: 9319, Loss: 0.0000028400
Train epoch: 9320, Loss: 0.0000030736
Train epoch: 9321, Loss: 0.0000026358
Train epoch: 9322, Loss: 0.0000033642
Train epoch: 9323, Loss: 0.0000030132
Train epoch: 9324, Loss: 0.0000028662
Train epoch: 9325, Loss: 0.0000023973
Train epoch: 9326, Loss: 0.0000025092
Train epoch: 9327, Loss: 0.0000026702
Train epoch: 9328, Loss: 0.0000026510
Train epoch: 9329, Loss: 0.0000031743
Train epoch: 9330, Loss: 0.0000028034
Train epoch: 9331, Loss: 0.0000026796
Train epoch: 9332, Loss: 0.0000026228
Train epoch: 9333, Loss: 0.0000024159
Train epoch: 9334, Loss: 0.0000025481
Train epoch: 9335, Loss: 0.0000025694
Train epoch: 9336, Loss: 0.0000026648
Train epoch: 9337, Loss: 0.0000027099
Train epoch: 9338, Loss: 0.0000028010
Train epoch: 9339, Loss: 0.0000028361
Train epoch: 9340, Loss: 0.0000024875
Train epoch: 9341, Loss: 0.0000027653
Train epoch: 9342, Loss: 0.0000027169
Train epoch: 9343, Loss: 0.0000030328
Train epoch: 9344, Loss: 0.0000025897
Train epoch:

Train epoch: 9535, Loss: 0.0000026037
Train epoch: 9536, Loss: 0.0000024809
Train epoch: 9537, Loss: 0.0000025924
Train epoch: 9538, Loss: 0.0000030062
Train epoch: 9539, Loss: 0.0000025385
Train epoch: 9540, Loss: 0.0000027274
Train epoch: 9541, Loss: 0.0000034553
Train epoch: 9542, Loss: 0.0000023647
Train epoch: 9543, Loss: 0.0000026141
Train epoch: 9544, Loss: 0.0000024654
Train epoch: 9545, Loss: 0.0000024611
Train epoch: 9546, Loss: 0.0000026356
Train epoch: 9547, Loss: 0.0000026681
Train epoch: 9548, Loss: 0.0000024122
Train epoch: 9549, Loss: 0.0000027868
Train epoch: 9550, Loss: 0.0000026028
Train epoch: 9551, Loss: 0.0000025019
Train epoch: 9552, Loss: 0.0000035298
Train epoch: 9553, Loss: 0.0000033207
Train epoch: 9554, Loss: 0.0000026555
Train epoch: 9555, Loss: 0.0000029570
Train epoch: 9556, Loss: 0.0000026463
Train epoch: 9557, Loss: 0.0000026678
Train epoch: 9558, Loss: 0.0000030728
Train epoch: 9559, Loss: 0.0000024263
Train epoch: 9560, Loss: 0.0000027059
Train epoch:

Train epoch: 9751, Loss: 0.0000023667
Train epoch: 9752, Loss: 0.0000025880
Train epoch: 9753, Loss: 0.0000024955
Train epoch: 9754, Loss: 0.0000026056
Train epoch: 9755, Loss: 0.0000026763
Train epoch: 9756, Loss: 0.0000028385
Train epoch: 9757, Loss: 0.0000026709
Train epoch: 9758, Loss: 0.0000029456
Train epoch: 9759, Loss: 0.0000029935
Train epoch: 9760, Loss: 0.0000025666
Train epoch: 9761, Loss: 0.0000027230
Train epoch: 9762, Loss: 0.0000032740
Train epoch: 9763, Loss: 0.0000025283
Train epoch: 9764, Loss: 0.0000023882
Train epoch: 9765, Loss: 0.0000030200
Train epoch: 9766, Loss: 0.0000028460
Train epoch: 9767, Loss: 0.0000027189
Train epoch: 9768, Loss: 0.0000024715
Train epoch: 9769, Loss: 0.0000026104
Train epoch: 9770, Loss: 0.0000027985
Train epoch: 9771, Loss: 0.0000027549
Train epoch: 9772, Loss: 0.0000023617
Train epoch: 9773, Loss: 0.0000029394
Train epoch: 9774, Loss: 0.0000025495
Train epoch: 9775, Loss: 0.0000030377
Train epoch: 9776, Loss: 0.0000027493
Train epoch:

Train epoch: 9967, Loss: 0.0000027228
Train epoch: 9968, Loss: 0.0000028444
Train epoch: 9969, Loss: 0.0000026674
Train epoch: 9970, Loss: 0.0000025282
Train epoch: 9971, Loss: 0.0000027987
Train epoch: 9972, Loss: 0.0000026334
Train epoch: 9973, Loss: 0.0000026280
Train epoch: 9974, Loss: 0.0000034806
Train epoch: 9975, Loss: 0.0000027250
Train epoch: 9976, Loss: 0.0000024398
Train epoch: 9977, Loss: 0.0000025953
Train epoch: 9978, Loss: 0.0000025588
Train epoch: 9979, Loss: 0.0000027401
Train epoch: 9980, Loss: 0.0000026571
Train epoch: 9981, Loss: 0.0000025229
Train epoch: 9982, Loss: 0.0000028392
Train epoch: 9983, Loss: 0.0000022147
Train epoch: 9984, Loss: 0.0000023455
Train epoch: 9985, Loss: 0.0000026847
Train epoch: 9986, Loss: 0.0000027564
Train epoch: 9987, Loss: 0.0000023537
Train epoch: 9988, Loss: 0.0000024356
Train epoch: 9989, Loss: 0.0000024428
Train epoch: 9990, Loss: 0.0000027499
Train epoch: 9991, Loss: 0.0000025691
Train epoch: 9992, Loss: 0.0000024645
Train epoch:

In [24]:
# PATH = './CNN2_SGDlr0.001_BCEL_EP10000_72%.pth'
# torch.save(net.state_dict(), PATH)

In [25]:
correct = 0
total = 0
nb_classes = 2
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for data in trainLoader:
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = net(images)
        predicted = torch.round(outputs)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
        

print('Accuracy of the network on the %s test images: %d %%' % (total,
    100 * correct / total))

print(confusion_matrix)

Accuracy of the network on the 383 test images: 100 %
tensor([[224.,   0.],
        [  0., 159.]])


In [26]:
correct = 0
total = 0
nb_classes = 2
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for data in testLoader:
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = net(images)
        predicted = torch.round(outputs)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
        

print('Accuracy of the network on the %s test images: %d %%' % (total,
    100 * correct / total))

print(confusion_matrix)

Accuracy of the network on the 96 test images: 75 %
tensor([[48.,  8.],
        [16., 24.]])


In [27]:
# PATH = './CNN2_SGDlr0.001_BCEL_EP10000_75%.pth'
# torch.save(net.state_dict(), PATH)